In [1]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal.fake_backends import LocalSimulator


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist
from PatchedMeasCal.bv import bv_circuit_cmap



from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import mthree

import random

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [2]:
def distance_measure(x, bv_value):
    if bv_value in x:
        return 1 - (x[bv_value] / sum(x.values()))
    return 1

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [3]:
n_qubits = 5
n_meas_shots = 16000
n_build_shots = 16000
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 
n_shots_qiskit_partial = n_build_shots // (n_qubits)


n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibmq_manila')

circuit = GHZ_prep(backend)

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

err_cmap = [[1, 3], [1, 4], [1, 2], [3, 4], [0, 1]]

## Mitigation Methods Build

In [4]:
full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
linear_filter = qiskit_linear(backend, n_qubits, n_shots_qiskit_partial)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
tpf_err.build(verbose=True)

tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
tpf.build(verbose=True)

ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f419e3151c0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f419e3151c0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002


Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s
Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s


## Run with repetitions
If everything above is working, let's crank it out a few times

In [5]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[],
    'm3':[]
}

for _ in range(n_reps):

    target_value = random.randint(0, 2 ** (n_qubits - 1) - 1)
    bv_value = bin((target_value << 1) + 1)[2:].zfill(n_qubits)[::-1]
    bv_string = bin(target_value)[2:].zfill(n_qubits - 1)
    circuit = bv_circuit_cmap(bv_string, n_qubits, backend)
    dist_measure = lambda x: distance_measure(x, bv_value)
    
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        dist_measure(bare_result)
    )
    results['full'].append(
        dist_measure(full_filter.apply(bare_result))
    )
    results['linear'].append(
        dist_measure(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
        dist_measure(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
        dist_measure(tpf_err.apply(bare_result))
    )
    
    results ['m3'].append(
        dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution())
    )
    
    results['aim'].append(dist_measure(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(dist_measure(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(dist_measure(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))
    

ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f41e8a06f70>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:cl

KeyboardInterrupt: 

In [6]:
results

{'bare': [0.40075000000000005,
  0.08118749999999997,
  0.3944375,
  0.484125,
  0.445875,
  0.20781249999999996,
  0.27618750000000003,
  0.359125,
  0.32743750000000005,
  0.4651875],
 'full': [0.16962635865295572,
  1.448682285243308e-10,
  0.20604847499268175,
  0.3563342675931813,
  0.24414011227660803,
  0.021165845906534586,
  0.03695250190914656,
  0.1471875274527904,
  0.20788800455632772,
  0.3899219438224062],
 'linear': [0.22291636992064756,
  5.575762074272461e-12,
  0.23413397907386768,
  0.3349337799118711,
  0.26885264436357437,
  0.060455409131074944,
  0.07990553047107118,
  0.16518264570094565,
  0.21822909061521834,
  0.3841139729699884],
 'aim': [0.29433333333333334,
  0.13916666666666666,
  0.30125,
  0.3805833333333334,
  0.35224999999999995,
  0.16337500000000005,
  0.2610416666666666,
  0.364,
  0.32366666666666666,
  0.3660416666666667],
 'sim': [0.33459375,
  0.16437500000000005,
  0.33728125,
  0.4255,
  0.381375,
  0.15993749999999995,
  0.27296875,
  0.357

In [7]:
bv_manila_results = {'bare': [0.40075000000000005,
  0.08118749999999997,
  0.3944375,
  0.484125,
  0.445875,
  0.20781249999999996,
  0.27618750000000003,
  0.359125,
  0.32743750000000005,
  0.4651875],
 'full': [0.16962635865295572,
  1.448682285243308e-10,
  0.20604847499268175,
  0.3563342675931813,
  0.24414011227660803,
  0.021165845906534586,
  0.03695250190914656,
  0.1471875274527904,
  0.20788800455632772,
  0.3899219438224062],
 'linear': [0.22291636992064756,
  5.575762074272461e-12,
  0.23413397907386768,
  0.3349337799118711,
  0.26885264436357437,
  0.060455409131074944,
  0.07990553047107118,
  0.16518264570094565,
  0.21822909061521834,
  0.3841139729699884],
 'aim': [0.29433333333333334,
  0.13916666666666666,
  0.30125,
  0.3805833333333334,
  0.35224999999999995,
  0.16337500000000005,
  0.2610416666666666,
  0.364,
  0.32366666666666666,
  0.3660416666666667],
 'sim': [0.33459375,
  0.16437500000000005,
  0.33728125,
  0.4255,
  0.381375,
  0.15993749999999995,
  0.27296875,
  0.35721875000000003,
  0.35478125000000005],
 'jigsaw': [0.17660219723121362,
  0.027017907390001317,
  0.2096981631900321,
  0.2533216368531074,
  0.2682199025059465,
  0.03891096744575562,
  0.10838538962434974,
  0.18312278026140982,
  0.1319007984686118],
 'cmc': [0.3095066672658163,
  0.03263611312391779,
  0.3029313390690618,
  0.3880624201460201,
  0.3690149737806875,
  0.09666550711663446,
  0.14885406249939626,
  0.27356210720574914,
  0.2454372282767645,
  0.3862087759658426],
 'cmc_err': [0.2544839547905904,
  0.01386139588659352,
  0.24380057315013848,
  0.39263995040337574,
  0.3567585556599886,
  0.08759289325857089,
  0.14196199605286497,
  0.24912557688975068,
  0.20504994540222876,
  0.3548367788383444],
 'm3': [0.22959193359255725,
  0.0,
  0.24195291012520692,
  0.32913242577657575,
  0.25899971471792405,
  0.07333322551956178,
  0.08588778383091078,
  0.14669541655570362,
  0.24827952196492353,
  0.38656130757116447]}

In [8]:
res = bv_manila_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')

bare & $0.34\substack{+0.14 \\ -0.26}$
full & $0.18\substack{+0.21 \\ -0.18}$
linear & $0.20\substack{+0.19 \\ -0.20}$
aim & $0.29\substack{+0.09 \\ -0.16}$
sim & $0.31\substack{+0.12 \\ -0.15}$
jigsaw & $0.16\substack{+0.11 \\ -0.13}$
cmc & $0.26\substack{+0.13 \\ -0.22}$
cmc_err & $0.23\substack{+0.16 \\ -0.22}$
m3 & $0.20\substack{+0.19 \\ -0.20}$
